In [25]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import copy
from keras.models import model_from_json
import os
from sklearn.preprocessing import LabelEncoder
from PIL import Image

## Part 2: Segementing license characters

In [ ]:
plate_image = cv2.imread("texas.jpg")
plt.imshow(plate_image)

In [ ]:
cropped = plate_image[25:69,20:250]
plt.imshow(cropped)

In [ ]:
# convert to grayscale and blur the image
image = cv2.cvtColor(cropped, cv2.COLOR_BGR2RGB)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blur = cv2.GaussianBlur(gray,(7,7),0)

# Applied inversed thresh_binary 
binary = cv2.threshold(blur, 180, 255,
                        cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

kernel3 = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
thre_mor = cv2.morphologyEx(binary, cv2.MORPH_DILATE, kernel3)

    
# visualize results    
fig = plt.figure(figsize=(12,7))
plt.rcParams.update({"font.size":18})
grid = gridspec.GridSpec(ncols=2,nrows=3,figure = fig)
plot_image = [plate_image, gray, blur, binary,thre_mor]
plot_name = ["plate_image","gray","blur","binary","dilation"]

for i in range(len(plot_image)):
    fig.add_subplot(grid[i])
    plt.axis(False)
    plt.title(plot_name[i])
    if i ==0:
        plt.imshow(plot_image[i])
    else:
        plt.imshow(plot_image[i],cmap="gray")


In [26]:
x = Image.fromarray(gray)
x.save("gray.jpg")

In [28]:
letters = os.popen("tesseract gray.jpg outputbase").read()

Estimating resolution as 497
Empty page!!
Estimating resolution as 497
Empty page!!
